In [ ]:
import random
import os
import pyaudio
import numpy as np
import json
import time
from pydub.playback import play
from pydub import AudioSegment
import serial

In [ ]:
def preAudio(audio, length, db):
    target_duration = length * 1000
    audio = audio[:target_duration]
    audio = np.array(audio.get_array_of_samples())
    audio = audio / audio.std(ddof=1)
    audio = (audio * db).astype(np.int16)
    return audio

In [ ]:
s = serial.Serial('com4', 115200, timeout=1)
s0 = [255]
s.write(s0)

start_male = AudioSegment.from_file('male.mp3', format='mp3')
start_female = AudioSegment.from_file('female.mp3', format='mp3')
end_male = AudioSegment.from_file('end_male.mp3', format='mp3')
end_female = AudioSegment.from_file('end_female.mp3', format='mp3')

male_audio = ['./Audio/male/' + i for i in os.listdir('./Audio/male')]
female_audio = ['./Audio/female/' + i for i in os.listdir('./Audio/female')]
random.shuffle(male_audio)
random.shuffle(female_audio)

In [ ]:
audio_length = 70  # s
db = 1000
log = []

name = input('please enter the name')
# for i in range(min(len(male_audio), len(female_audio)))
for i in range(5):
    male_i = AudioSegment.from_file(male_audio[i], format='mp3')
    female_i = AudioSegment.from_file(female_audio[i], format='mp3')

    # 处理音频文件
    male = preAudio(male_i, audio_length, db)
    female = preAudio(female_i, audio_length, db)
    
    # 随机规则，随机男女，随机左右，播放提示音
    att = random.sample(['male', 'female'], 1)[0]
    att_location = random.sample(['left', 'right'], 1)[0]
    
    trigger = []
    if 'male' == att and 'left' == att_location:
        trigger = [1]
    elif 'male' == att and 'right' == att_location:
        trigger = [3]
    elif 'female' == att and 'left' == att_location:
        trigger = [2]
    elif 'female' == att and 'right' == att_location:
        trigger = [4]
    
    log_i = {
        'i': i,
        'length': audio_length,
        'male_i': male_audio[i],
        'female_i': female_audio[i],
        'att': att,
        'att_location': att_location,
        'trigger': trigger[0]
    }
    print(log_i)
    trigger = [s0[0] - trigger[0]]
    
    if 'male' == att:
        play(start_male)
    else:
        play(start_female)
    time.sleep(1)
    
    # 播放音频，并向saga发送triggers
    stream = pyaudio.PyAudio().open(format=pyaudio.paInt16, channels=2, rate=48000, output=True)
    
    audio_data = np.array([])
    if ('male' == att and 'left' == att_location) or ('female' == att and 'right' == att_location):
        audio_data = np.column_stack((male, female)).flatten()
    elif ('male' == att and 'right' == att_location) or ('female' == att and 'left' == att_location):
        audio_data = np.column_stack((female, male)).flatten()
    
    s.write(trigger)
    stream.write(audio_data.tobytes())
    s.write(s0)
    stream.stop_stream()
    stream.close()
    time.sleep(1)
    
    if 'male' == att:
        play(end_male)
    else:
        play(end_female)
    
    # 记录日志
    log.append(log_i)
    input('press any key to continue')
    time.sleep(1)

s.close()
log = json.dumps(log)
with open('./log/{}-{}.json'.format(name, int(time.time())), 'w+', encoding='utf-8') as file:
    file.write(log)
print('end')